In [ ]:
import numpy as np
import wurtzite as wzt
import matplotlib.pyplot as plt
import matplotlib

## 1st dislocation

In [ ]:
l0 = wzt.generate.create_lattice(
    dimensions=(5, 5, 3),  #  The number of cells
    cell="B4_ZnS",
)

In [ ]:
wzt.visualization.render(molecule=l0, window_size=(800, 800))

In [ ]:
b0 = [1, 0, 0]
position0 = [5, 6, 7.5]
plane0 = (0, 0, 1)

u0 = wzt.dislocations.displace_love(
    crystal=l0,
    position=position0,
    burgers_vector=b0,
    plane=plane0,
    bv_fraction=1.0,
)

In [ ]:
rt = wzt.dislocations._get_rotation_tensor(
        burgers_vector=b0,
        plane=plane0,
        cell=l0.cell
)
rt

In [ ]:
%matplotlib widget
plt.xlabel("OX ($\AA$)")
plt.ylabel("OY ($\AA$)")
plt.title("Dislocation field (u), z = 0 projection")
y_dim = 1
x_dim = 0

plt.quiver(l0.coordinates[..., x_dim], l0.coordinates[..., y_dim], u0[..., x_dim], u0[..., y_dim], 
           color=wzt.visualization.vectors_to_rgb(u0[..., (x_dim, y_dim)]))
plt.scatter(position0[x_dim], position0[y_dim], s=200.5, c="brown", marker="1")
plt.tight_layout()
plt.gca().invert_yaxis()

In [ ]:
l1 = l0.translate(u0)
l1 = wzt.generate.update_bonds(l1)
wzt.visualization.render(molecule=l1, window_size=(800, 800))

In [ ]:
def beta(burgers_vector, bv_fraction, plane, cell, x):
    rt = wzt.dislocations._get_rotation_tensor(
        burgers_vector=burgers_vector,
        plane=plane,
        cell=cell
    )
    burgers_vector = bv_fraction*cell.to_cartesian_indices(burgers_vector)
    burgers_vector = burgers_vector.reshape(-1, 1)
    burgers_vector = rt.dot(burgers_vector).squeeze()

    be = np.sqrt(burgers_vector[0]**2 + burgers_vector[1]**2)
    bz = burgers_vector[2]
    return rt, wzt.dislocations.beta_function(x, be=be, bz=bz)